In [1]:
import gurobipy as gp
from gurobipy import GRB

## Parameters

In [2]:
arcs, costs = gp.multidict({
    ('Denver', 'Kansas'): 2,
    ('Denver', 'Louisville'): 3,
    #
    ('Atlanta', 'Kansas'): 3,
    ('Atlanta', 'Louisville'): 1,
    #
    ('Kansas', 'Detroit'): 2,
    ('Kansas', 'Miami'): 6,
    ('Kansas', 'Dallas'): 3,
    ('Kansas', 'NewOrleans'): 6,
    #
    ('Louisville', 'Detroit'): 4,
    ('Louisville', 'Miami'): 4,
    ('Louisville', 'Dallas'): 6,
    ('Louisville', 'NewOrleans'): 5,
})

supply = {
    'Denver': 600,
    'Atlanta': 400,

}

demand = {
    'Kansas': 0, 
    'Louisville':0,
    'Detroit': 200,
    'Miami': 150,
    'Dallas':350, 
    'NewOrleans': 300,
}

# Model
## Decision Variables

In [3]:
m = gp.Model('transshipment')
send = m.addVars(arcs, vtype=GRB.CONTINUOUS, name='send')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-19


## Objective Function

In [4]:
Z = send.prod(costs)
m.ModelSense = GRB.MINIMIZE
m.setObjective(Z)

## Constraints

In [5]:
source = m.addConstrs(
    (send.sum(i,'*') <= s for i,s in supply.items()),
    name='source'
)

sinks = m.addConstrs(
    # Out - In = - D  <--> In - Out = D
    (send.sum('*',k) - send.sum(k, '*')  == d for k,d in demand.items()),
    name='sinks'
)

In [6]:
m.update()
print(f'Number of variables: {m.NumVars}')
print(f'Number of constraints: {m.NumConstrs}')
print(f'Constraints are \n {m.getConstrs()} \n')

Number of variables: 12
Number of constraints: 8
Constraints are 
 [<gurobi.Constr source[Denver,600]>, <gurobi.Constr source[Atlanta,400]>, <gurobi.Constr sinks[Kansas,0]>, <gurobi.Constr sinks[Louisville,0]>, <gurobi.Constr sinks[Detroit,200]>, <gurobi.Constr sinks[Miami,150]>, <gurobi.Constr sinks[Dallas,350]>, <gurobi.Constr sinks[NewOrleans,300]>] 



## Optimization process

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x650e0c54
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 4 rows, 8 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1500000e+03   2.312500e+02   0.000000e+00      0s
       3    5.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.200000000e+03


## Results

In [8]:
print(f'Objective value ${m.objVal:,.2f}')

for v in m.getVars():
    if abs(v.X) > 0.000001:
        print(f'{v.VarName}: {v.X}')

Objective value $5,200.00
send[Denver,Kansas]: 600.0
send[Atlanta,Louisville]: 400.0
send[Kansas,Detroit]: 200.0
send[Kansas,Dallas]: 350.0
send[Kansas,NewOrleans]: 50.0
send[Louisville,Miami]: 150.0
send[Louisville,NewOrleans]: 250.0
